# Modify the separable net for simple crf

This modifies the seprable/low-rank-filter classifier adding inputs from a previous pass of inference. 
The idea is that you could do this:

```python
initial_net.forward(data=array([rgb]))
probs = [initial_net.blob[prob_feature].data[0,(0,2,3)] for prob_feature in FEATURES]
for pass in range(npasses):
    results = net.forward(data=array([concatenate([rgb, probs])]))
    probs = [net.blob[prob_feature].data[0,(0,2,3)] for prob_feature in FEATURES]
```
assuming that `net` is a trained net, that `FEATURES` is a list of output probability layer names for each feature.  Then the final result would be similar to CRF optimizatio,


Before using this you should:
- Have pycaffe (from https://github.com/alexgkendall/caffe-segnet) setup and on the path
- Have a copy of solver.prototxt
- Have a copy of the modified segnet training classifier (e.g. `modified-training-net.prototxt`)
- Have a copy of the 12-target segnet inference classifier (e.g. `modified-inference-net.prototxt`)

Table of contents:
- [Utility Functions](#Utility-Functions)
- [Add Low Rank Filter (Function)](#Add-Low-Rank-Filters)
- [Generate the Training Net](#Generate-the-net-to-use-for-training)
- [Generate the Inference Net](#Generate-the-net-to-use-for-inference-/-testing)
- [Test the Net](#Final-Test)
- [Modify the Solver Prototxt](#Set-the-solver-parameters-to-use-the-new-net)

# Utility Functions

In [ ]:
%pylab notebook

In [ ]:
import os
import google.protobuf.text_format

In [ ]:
import caffe
import caffe.proto.caffe_pb2

In [ ]:
# For debugging (cell can be removed from final notebook)
from IPython.core.debugger import Tracer
set_trace = Tracer()

In [ ]:
def read_net_proto(path):
    """Read a net from a prototxt file
    
    :param path: The path to a caffe network (.prototxt file)
    
    :return: The prototxt object
    :rtype: caffe.proto.caffe_pb2.NetParameter
    """
    net = caffe.proto.caffe_pb2.NetParameter()
    with open(path) as f:
        proto = f.read()
    google.protobuf.text_format.Parse(proto, net)
    return net
    

In [ ]:
def summarize_net(train_net):
    """ Print a (not sooo short) summary of the net layers
    
    :param net: A net 
    :type net: caffe.proto.caffe_pb2.NetParameter
    """
    layers = list(train_net.layer)
    for i, layer in enumerate(layers):
        print "{:04}".format(i),
        print "\t{:15}\t{:15}".format(layer.name, layer.type),
        if layer.type=="Convolution":
            if layer.convolution_param.kernel_size > 0:
                print "\t{0:>2}x{0:<2}".format(layer.convolution_param.kernel_size),
            else:
                print "\t{:>2}x{:<2}".format(layer.convolution_param.kernel_w, layer.convolution_param.kernel_h),                
        else:
            print "\t{:5}".format(''),

        if "_D" in layer.name:
            print "DECODE"
        else:
            print "      "

    print "Total", len(layers), "layers"   

In [ ]:
def save_net_proto(path, net):
    new_proto = google.protobuf.text_format.MessageToString(net)
    with open(path, 'w') as f:
        f.write(new_proto)

# Modify the inference net to consume output

In [ ]:
FEATURES = ['facade', 'window', 'door', 'cornice', 'sill', 'balcony', 'blind', 'deco', 'molding', 'pillar', 'shop']
NUM_FEATURES = len(FEATURES)
NEG = NEGATIVE = 0
UNK = UNKNOWN = 1
POS = POSITIVE = 2
EDG = EDGE = 3

In [ ]:
#inference_net = read_net_proto('modified-inference-net.prototxt')
inference_net = read_net_proto('non-bayesian-inference-net.prototxt')

In [ ]:
print inference_net.input_dim

In [ ]:
inference_net.input_dim[0] = 1 # Batch size of one, since batches are bigger
inference_net.input_dim[1] = 3 + NUM_FEATURES*3  # (R,G,B + NUM_FEATURES*(NEG,POS,EDG))
print inference_net.input_dim

In [ ]:
save_net_proto('crf_inference_net.prototxt', inference_net) 

The only real difference should be in the number of input channels (line 3) that it expects

In [ ]:
!head crf_inference_net.prototxt -n 5 | cat --number

# Modify the training net to consume its own output


In [ ]:
train_net = read_net_proto('modified-training-net.prototxt')

In [ ]:
orig_net = train_net

new_net= caffe.proto.caffe_pb2.NetParameter()
new_net.CopyFrom(orig_net)
layers = new_net.layer._values

In [ ]:
data_layer = layers[0]
data_layer.python_param.module = "python_layers_with_prior"
data_layer.python_param.layer = "TrainInputLayerWithPrior"

In [ ]:
print data_layer

In [ ]:
save_net_proto('crf_training_net.prototxt', new_net)

# Make the new python layer for training...

I split the python input layer into two files:
- [prepare_crf_input.py](/edit/scripts/anisotropic-training/prepare_crf_input.py) -- Uses the inference net to get labels using the non-crf network.
- [python_layers_with_prior.py](/edit/scripts/anisotropic-training/python_layers_with_prior.py) -- Feeds a network with the best non-crf estimate of the labels as additional inputs. The idea is that we could use a net trained this way in a loop...

# Transfer the weights in

In [ ]:
import caffe
caffe.set_mode_cpu()  # No need to consume CPU for this...

In [ ]:
import warnings
with warnings.catch_warnings('ignore'):
    old_net = caffe.Net('modified-training-net.prototxt', 'deploy/test_weights.caffemodel', caffe.TEST)
    new_net = caffe.Net('crf_training_net.prototxt', caffe.TEST)

In [ ]:
from scipy.stats import truncnorm
truncated_normal = truncnorm(-1, 1).rvs

In [ ]:
for param in old_net.params:
    if param == 'conv1_1':
        N,C,H,W = new_net.params['conv1_1'][0].data.shape
        new_net.params['conv1_1'][0].data[:, :3, :, :] = old_net.params['conv1_1'][0].data[...]
        new_net.params['conv1_1'][0].data[:, 3:, :, :] = truncated_normal(size=(N, C-3, H, W))
        new_net.params['conv1_1'][1].data[...] = old_net.params['conv1_1'][1].data[...]
    else:
        for i in range(len(old_net.params[param])):
            new_net.params[param][i].data[...] = old_net.params[param][i].data

In [ ]:
new_net.save('crf_initial_weights.caffemodel')

# Set the solver parameters to use the new net

In [ ]:
import caffe
import os
import google.protobuf.text_format

In [ ]:
solver = caffe.proto.caffe_pb2.SolverParameter()
google.protobuf.text_format.Merge(open('solver.prototxt').read(), solver);

In [ ]:
solver.net = os.path.abspath('crf_training_net.prototxt')

In [ ]:
solver.snapshot_prefix = os.path.join(os.path.dirname(solver.snapshot_prefix), "crf_facades")

In [ ]:
!mkdir -p {os.path.dirname(solver.snapshot_prefix)}

In [ ]:
print solver

In [ ]:
with open('crf_solver.prototxt', 'w') as f:
    f.write(google.protobuf.text_format.MessageToString(solver))

# Make sure the GPU is ready

In [ ]:
old_net = None; del old_net
new_net = None; del new_net

In [ ]:
!nvidia-smi

_I_ am not using the GPU, but there may be some zombie processes claiming some of its RAM

In [ ]:
print os.getpid()

In [ ]:
# !kill 27026
# !kill 6354
# !kill 17722

# Training

For this training loop, I need to save the outputs after each iteration. 

In [ ]:
%pylab notebook

In [ ]:
import caffe
import os
caffe.set_mode_gpu()
caffe.set_device(0)

In [ ]:
import anydbm
import json

history = anydbm.open('crf_training_history', 'c')

iteration = len(history)
print iteration

In [ ]:
with warnings.catch_warnings('ignore'):
    solver = caffe.SGDSolver('crf_solver.prototxt')

In [ ]:
iteration

In [ ]:
if iteration == 0:
    solver.net.copy_from('crf_initial_weights.caffemodel')
else:
    iteration = iteration - iteration % 1000
    snapshot = '/home/shared/Projects/Facades/mybook/anisotropic/crf_facades_iter_{}.solverstate'
    snapshot = snapshot.format(iteration)
    solver.restore(snapshot)

In [ ]:
solver.net.layers[0].verbose = False
solver.net.epochs = iteration % len(solver.net.layers[0].files)

In [ ]:

def softmax(a, axis=0):
    a = np.exp(a - a.max(axis=axis))
    a /= a.sum(axis=axis)
    return a

In [ ]:
#from pyfacades.util import softmax
#  ^--- my softmax was not normalizing properly...


def visualize_progress(fig, prior=None, prior_loss=0):
    #imshow(solver.net.blobs['concat'].data[0, 6])
    fig.clf()
    subplot(221)
    imshow(solver.net.blobs['data'].data[0,:3].transpose(1,2,0)/255.)
    xticks([]);yticks([]);xlabel('rgb')
    subplot(222)
    imshow(solver.net.blobs['window'].data[0,0])  # Color(3) + Windows(1)*NumLabels(3) + Positive(1)
    xticks([]);yticks([]);xlabel('expected')
    subplot(223)
    if prior is not None:
        imshow(prior)  # Color(3) + Windows(1)*NumLabels(3) + Positive(1)
        xticks([]);yticks([]);xlabel('intitial({:.3f})'.format(prior_loss))
    subplot(224)
    imshow(softmax(solver.net.blobs['conv-window'].data[0])[2])
    current_loss = float(solver.net.blobs['window-loss'].data)
    xticks([]);yticks([]);xlabel('current({:.3f})'.format(current_loss))
    #fig.canvas.draw()


In [ ]:
from munch import Munch
def log_image():
    record = Munch()
    input_layer  = solver.net.layers[0]
    record.time = datetime.datetime.isoformat(datetime.datetime.now())
    record.epoch = input_layer.epochs
    record.image = input_layer.files[input_layer.counter]
    record.loss = {loss_layer.replace('-loss',''):float(solver.net.blobs[loss_layer].data) for loss_layer in solver.net.outputs}
    record.total_loss = sum(record.loss.values())
    record.iteration = iteration
    
    history[json.dumps(iteration)] = json.dumps(record)
    
    return record

In [ ]:
from munch import munchify
def get_log_record(i):
    return munchify(json.loads(history[json.dumps(i)]))

In [ ]:
print get_log_record(100).loss.window

In [ ]:
def get_last_losses(n=1000):
    n = min(n, iteration)
    iterations = arange(iteration-n, iteration)
    losses = [get_log_record(i).loss.window for i in iterations]       
    return iterations, losses

In [ ]:
from scipy.signal import gaussian, convolve

FILTER_HWIDTH=50
FILTER =  gaussian(2*FILTER_HWIDTH+1, 25)
FILTER /= sum(FILTER)

def plot_last_losses(n=1000, ax = None):
    ax = ax or gca()
    i, loss = get_last_losses(n)
    ax.plot(i, loss, alpha=0.2, c='red')
    ax.plot(i[FILTER_HWIDTH:-FILTER_HWIDTH],  convolve(loss, FILTER, mode='valid'))
    ax.set_xlim(i[0], i[-1])

## Training Loop
The cell below executes the training loop. This can take a LOONG time, so I do not want to execute it by accident as I work through the notebook. 

I have disabled the cell by marking it as a `raw` cell in jupyter. In order to train the net you need to convert the next cell to `python` again. 

In [ ]:
print "We are currently at iteration", iteration

In [ ]:
FEATURES = ['facade', 'window', 'door', 'cornice', 'sill', 'balcony', 'blind', 'deco', 'molding', 'pillar', 'shop']

In [ ]:
import python_layers_with_prior
reload(python_layers_with_prior)

In [ ]:
solver.net.layers[0].epochs = iteration / len(solver.net.layers[0].files)

In [ ]:
%pdb on

In [ ]:
fig = figure(figsize=(12,5))

for i in range(1000000):
    solver.net.layers[0].priors = None
    for j in range(10):
        solver.step(1)
        if j == 0:
            prior = solver.net.blobs['data'].data[0, 2*3+1].copy()
            prior_loss = float(solver.net.blobs['window-loss'].data)
        probs = [softmax(solver.net.blobs['conv-{}'.format(feature)].data[0,(0,2,3)]) for feature in FEATURES]
        probs = np.concatenate(probs, axis=0)
        solver.net.layers[0].priors = probs
        visualize_progress(fig, prior, prior_loss)
        suptitle('step {}, epoch {}, image {}, iter {}'.format(i, solver.net.layers[0].epochs,solver.net.layers[0].counter,  j))
        fig.canvas.draw()
    log_image()
    iteration += 1

In [ ]:
figure(frameon=False)
plot_last_losses(1e10)

#  Compute the batch normalization
This competes with training for access to the GPU -- best restart the Kernel first

> **ATTENTION:** _You need to restart the kernel here before proceeding!_

The GPU does not have enough RAM to hold the models for both BN and training.

In [ ]:
%pylab notebook
import anydbm
import json

history = anydbm.open('crf_training_history', 'c')
iteration = len(history)

In [ ]:
!nvidia-smi

In [ ]:
%%bash --out model_file 
./get_iter.py -m -p crf_solver.prototxt

In [ ]:
model_file = model_file.strip()
print model_file

> **WARNING:** The next cell takes about 90 minutes to complete
So I have commented it out -- you will need to uncomment it to run it again. 

In [ ]:
!mkdir -p crf-deploy
%run compute_bn_statistics.py crf_training_net.prototxt {model_file.strip()}  crf-deploy

For whatever reason, the notebook is a bit.. kludgy after the BN procedure finishes. I suspect that I am leaking some resource when I do all of the plotting, or perhaps I produce to much output. 

You may need to comment out line 2 in the cell above and re-run the cell up to this point.

# Qualitatively Evaluate the net (after applying BN)

Now we should be able to load a crf inference net

> **NOTE:** After running BN it is a good idea to restart the kernel, etc. and make sure that the GPU RAM is not being hogged...

In [ ]:
%pylab notebook
!nvidia-smi

In [ ]:
CPU = False

import caffe
if CPU:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0)

In [ ]:
net = caffe.Net('crf_inference_net.prototxt', 'crf-deploy/test_weights.caffemodel', caffe.TEST)

In [ ]:
EVAL_FILES = [fn.strip() for fn in open('./data/training/independant_12_layers/fold_01/eval.txt')]

In [ ]:
# %load prepare_crf_input.py

FEATURES = ['facade', 'window', 'door', 'cornice', 'sill', 'balcony', 'blind', 'deco', 'molding', 'pillar', 'shop']
WEIGHT = 'deploy/test_weights.caffemodel'
LAYOUT = 'modified-inference-net.prototxt'

NEG = NEGATIVE = 0
POS = POSITIVE = 2
EDG = EDGE = 3

import numpy as np
import caffe
init_net = caffe.Net(LAYOUT, WEIGHT, caffe.TEST)

# Set the batch size to one
init_net.blobs['data'].reshape(1, 3, 512, 512)
init_net.reshape()

def priors(im):
    """
    :param im: An input image, shape 3x512x512
    :type im: np.ndarray
    """
    result = init_net.forward(data=np.array([im[:3]]))
    probs = [init_net.blobs['prob-{}'.format(feature)].data[0,(NEG,POS,EDG)] for feature in FEATURES]
    probs = np.concatenate(probs, axis=0)
    return probs

def iterate(net):
    probs = [net.blobs['prob-{}'.format(feature)].data[0,(NEG,POS,EDG)] for feature in FEATURES]
    probs = np.concatenate(probs, axis=0)
    net.blobs['data'].data[0,3:] = probs
    result = net.forward(data=net.blobs['data'].data)
    return result
    
def prepare(im, prior=None):
    if prior is None:
        prior = priors(im)
    assert prior.shape == (len(FEATURES)*3, 512, 512)
    concat = np.concatenate([im[:3], prior])
    return np.array([concat])

In [ ]:
def plot_iter(fig=None):
    fig = fig or figure(figsize=(10,4))
    subplot(131)
    imshow(net.blobs['data'].data[0,:3].transpose(1,2,0)/255.)
    axis('off')
    subplot(132)
    imshow(net.blobs['data'].data[0,:3].transpose(1,2,0)/255.)
    imshow(net.blobs['prob-window'].data[0,2], alpha=0.65)
    axis('off')
    subplot(133)
    imshow(net.blobs['prob-window'].data[0,2])
    axis('off')
    tight_layout()
    fig.canvas.draw()

In [ ]:
import os

In [ ]:
idx = 24
data_with_targets = np.load(EVAL_FILES[idx])
data, targets = data_with_targets[:3], data_with_targets[3:]

In [ ]:
dirname = 'frame-image-{}'.format(idx)
!mkdir -p {dirname}

results = net.forward(data=prepare(data))
fig = figure(figsize=(10,4))
for i in range(1000):
    iterate(net)
    title('Iteratin {}'.format(i))
    plot_iter(fig)
    savefig(os.path.join(dirname, 'frame_{:05}'.format(i+1)))

In [ ]:
!ffmpeg -i frame-image-{idx}/frame_%05d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p  video-image-{idx}.mp4

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()

In [ ]:
results = iterate(net)
plot_iter()